<a href="https://colab.research.google.com/github/Maioy97/Clothes-Classification/blob/main/clothes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading dataset



In [ ]:
!pip install kaggle
# then upload kaggle.json
!pip install tensorflow
!pip install keras_flops
!pip install tqdm
!pip install opencv-python

     |████████████████████████████████| 462 kB 5.5 MB/s 


In [ ]:
!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json

In [ ]:
# !ls /root/.kaggle/
!pwd

/content


In [ ]:
!kaggle datasets download -d salil007/caavo
!unzip ./caavo.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/train/770_9.jpg   
  inflating: train/train/7710_2.jpg  
  inflating: train/train/7710_3.jpg  
  inflating: train/train/7710_4.jpg  
  inflating: train/train/7711_2.jpg  
  inflating: train/train/7711_3.jpg  
  inflating: train/train/7711_4.jpg  
  inflating: train/train/7712_2.jpg  
  inflating: train/train/7712_3.jpg  
  inflating: train/train/7712_4.jpg  
  inflating: train/train/7713_2.jpg  
  inflating: train/train/7713_3.jpg  
  inflating: train/train/7713_4.jpg  
  inflating: train/train/7714_2.jpg  
  inflating: train/train/7714_3.jpg  
  inflating: train/train/7714_4.jpg  
  inflating: train/train/7715_2.jpg  
  inflating: train/train/7715_3.jpg  
  inflating: train/train/7715_4.jpg  
  inflating: train/train/7716_2.jpg  
  inflating: train/train/7716_3.jpg  
  inflating: train/train/7716_4.jpg  
  inflating: train/train/7717_2.jpg  
  inflating: train/train/7717_3.jpg  
  inflating: train/train/7717_4.jpg  

In [ ]:
import os
import csv
#imports 
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.optimizers import Adam
from keras_flops import get_flops
import cv2
from tqdm import tqdm 
import numpy as np 
from sklearn.model_selection import train_test_split


##display class counts 

In [ ]:

root_path = "/content/train/train"
# train_file_names = os.walk("/content/train/train/")

# class_names = ["blouse", "cloak", "coat", "jacket", "dress", "t-shirt",
#                "workclothes", "shirts", "summer dresses", "suits", "sweater",
#                "printed_t-shirt", "undergarment","army","vest"]
def dataset_exploration(root_path): 
    train_file_names = os.listdir(root_path)
    train_file_names.sort()
    print(train_file_names)
    class_counts = [0] * 15
    train_csv = [["number", '0', '1', '2', '3', '4', '5', '6',
                  '7', '8', '9', '10', '11', '12', '13', '14']]

    past_row_num = -1
    current_list = [0] * 16

    # go through the train data file
    for name in train_file_names:
        # rowindex_categories.jpg
        name, extension = name.split('.')
        row_num, class_num = name.split('_')
        # list instances per class
        class_counts[int(class_num)] += 1
        # list categories per row in a csv
        if past_row_num != -1:
            if past_row_num == row_num:
                current_list[int(class_num) + 1] += 1
            else:
                train_csv.append(current_list)
                current_list = [0] * 16
                current_list[0] = row_num
                current_list[int(class_num) + 1] += 1
        else:
            current_list[0] = int(row_num)

        past_row_num = row_num


    #saving train image names 
    with open('train_file_list.txt', 'w') as file_:
        for name in train_file_names : 
            file_.write(f'{name}\n')
      
    with open('class_counts.txt', 'w') as file:
        for i in range(0,len(class_counts)):
            file.write(f'class number {i} : {class_counts[i]} \n')
    
        # not really needed , exploring something 
    with open('train_counts.csv', 'w') as f:
        writer = csv.writer(f)
          
        writer.writerow(train_csv[0])
        writer.writerows(train_csv[1::])

dataset_exploration(root_path)

['1000_1.jpg', '1000_10.jpg', '1000_11.jpg', '1000_12.jpg', '1000_13.jpg', '1000_2.jpg', '1000_3.jpg', '1000_4.jpg', '1000_6.jpg', '1000_7.jpg', '1000_8.jpg', '1000_9.jpg', '1001_1.jpg', '1001_10.jpg', '1001_11.jpg', '1001_12.jpg', '1001_13.jpg', '1001_2.jpg', '1001_3.jpg', '1001_4.jpg', '1001_6.jpg', '1001_7.jpg', '1001_8.jpg', '1001_9.jpg', '1002_1.jpg', '1002_10.jpg', '1002_11.jpg', '1002_12.jpg', '1002_13.jpg', '1002_2.jpg', '1002_3.jpg', '1002_4.jpg', '1002_6.jpg', '1002_7.jpg', '1002_8.jpg', '1002_9.jpg', '1003_1.jpg', '1003_10.jpg', '1003_11.jpg', '1003_12.jpg', '1003_13.jpg', '1003_2.jpg', '1003_3.jpg', '1003_4.jpg', '1003_6.jpg', '1003_7.jpg', '1003_8.jpg', '1003_9.jpg', '1004_1.jpg', '1004_10.jpg', '1004_11.jpg', '1004_12.jpg', '1004_13.jpg', '1004_2.jpg', '1004_3.jpg', '1004_4.jpg', '1004_6.jpg', '1004_7.jpg', '1004_8.jpg', '1004_9.jpg', '1005_1.jpg', '1005_10.jpg', '1005_11.jpg', '1005_12.jpg', '1005_13.jpg', '1005_2.jpg', '1005_3.jpg', '1005_4.jpg', '1005_6.jpg', '1005_7.j

In [ ]:
# maybe add data augmentation later (data isn't balanced)

##Resnet 50


##data reading

1. read train data
  (filename | category)
2. seperate into train valid 

In [ ]:
def get_labels_from_image_names(image_name_list):
    '''
    extracts the labels from the image names 
    image names are formatted the following way
    rownum_classnum.jpg
    :param image_name_list: list of image names that follow the dataset format  
    '''
    labels = []
    for image_name in image_name_list:
        row_num, label = image_name.split('.')[0].split('_')
        labels.append(int(label))
    return labels
      
def read_train_file_list(image_list_path):
    '''
    used to read full list of images in train or test folders
    list saved in a previouis step 
    :param image_list_path: path to the text file continaing the list of image names 
    '''
    with open(image_list_path) as f:
        lines = f.read().splitlines()
    return lines

def read_images(root_path, file_name_list, width, height):
    '''
    used to read the val data
    root_path: path to the folder where the images are present
    file_name_list: list with image names 
    :param width: width used to resize the images
    :param height: height used to resize the images
    '''
    image_list =np.zeros((len(file_name_list), width, height, 3),dtype=np.int32)
    image_num = 0
    # image_list = []
    for image_name in tqdm(file_name_list):
        tmp_im = cv2.imread(os.path.join(root_path, image_name))
        tmp_im = cv2.resize(tmp_im, (width, height))
        image_list[image_num]= tmp_im
        image_num += 1

    # label_list = get_labels_from_image_names(file_name_list)
    return image_list  #, label_list



In [ ]:
def training_data_genrator(image_name_list, batch_size, width, height, root_path):
    '''
    data generator that yeilds the current batch of data 
    should be given to the model.fit function 
    :param image_name_list: [(str)] list of image names 
    :param batch_size: training batch size
    :param width: (int) new wanted width after resize
    :param height: (int) new wanted height after resize
    :param root_path: (str) path to the folder containing the images listed in image_name_list
    '''

    while True : 
        for start in range(0, len(image_name_list), batch_size): 
          batch_row = 0 
          batch_data = np.zeros((batch_size, width, height, 3),dtype=np.int32)
          batch_labels = np.zeros((batch_size, 15), dtype=np.int32)
          end = min(start+ batch_size, len(image_name_list))
          for index in range(start, end):
              
              image_name = image_name_list[index]
              # remove extension and get the part after the _ (class number)
              label = int(image_name.split('.')[0].split('_')[1])
              label = keras.utils.to_categorical(label, num_classes=15)
              image = cv2.imread(os.path.join(root_path, image_name))
              image = cv2.resize(image, (width, height))
              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
              batch_data[batch_row] = image
              batch_labels[batch_row] = label

              batch_row += 1
          # print("batch_labels shape: ",np.shape(batch_labels))
          # print("batch_data shape: ",np.shape(batch_data))

          yield batch_data, batch_labels
              


In [ ]:
folder_path = "/content/train/train"
train_file_list = os.listdir(folder_path)
train_file_list.sort()
labels = get_labels_from_image_names(train_file_list)


In [ ]:
train_names, valid_names, train_labels, valid_labels = train_test_split(
                                                          train_file_list,
                                                          labels,
                                                          test_size=0.2,
                                                          random_state=42)


In [ ]:
width = 180 
height = 180
valid_images = read_images(folder_path, valid_names, width, height)
valid_labels = keras.utils.to_categorical(valid_labels) #, num_classes=15)
print(np.shape(valid_labels))
# valid_images, labels = read_images(root_path, valid_files_list, width, height)
# image_list=[]

100%|██████████| 12452/12452 [00:15<00:00, 820.65it/s] 


#creating model 




create model / load pretrained 
train (from scratch or from pretrained weights) to fit classes in the dataset 
set up metric drawing (history)  before training begins 


In [ ]:
# loading pretrained model 
resnet_model = Sequential()

pretrained_model= keras.applications.ResNet50(include_top=False,
                                              # 180, 180, 3
                                              input_shape=(width,height, 3), 
                                              pooling='avg',
                                              classes=15,
                                              weights='imagenet')

pretrained_model.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(layers.Dense(1024, activation='relu'))
resnet_model.add(layers.Dense(15, activation='softmax'))

resnet_model.summary()
flops = get_flops(resnet_model, batch_size=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 15)                15375     
                                                                 
Total params: 25,701,263
Trainable params: 2,113,551
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
flops

5442840986

In [ ]:
batch_size = 64
train_generator = training_data_genrator(train_names, batch_size,
                                         width, height, folder_path)
steps = len(train_names) // batch_size


#retraining 

# loss_fn = keras.losses.CategoricalCrossentropy()
resnet_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='categorical_crossentropy', #loss_fn,
                     metrics=['accuracy', 'Precision', 'Recall'])


history = resnet_model.fit(train_generator, 
                                  validation_data=(valid_images, valid_labels),
                                  epochs =50)
#, steps_per_epoch=steps)

Epoch 1/50
    334/Unknown - 2449s 7s/step - loss: 1.7061 - accuracy: 0.4606 - precision: 0.6596 - recall: 0.2721

In [ ]:
keras.utils.plot_model(resnet_model)

In [ ]:
from matplotlib import pyplot as plt

fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

##inception v3


In [ ]:
# loading pretrained model 
inception_model = Sequential()

pretrained_model= keras.applications.InceptionV3(include_top=False,
                                              # 180, 180, 3
                                              input_shape=(width,height, 3), 
                                              pooling='avg',
                                              classes=15,
                                              weights='imagenet')

pretrained_model.trainable = False

inception_model.add(pretrained_model)
inception_model.add(layers.Flatten())
inception_model.add(layers.Dense(1024, activation='relu'))
inception_model.add(layers.Dense(15, activation='softmax'))

inception_model.summary()
flops = get_flops(inception_model, batch_size=1)

In [ ]:
flops

In [ ]:
batch_size = 64
train_generatorI = training_data_genrator(train_names, batch_size,
                                         width, height, folder_path)
steps = len(train_names) // batch_size

valid_labels = keras.utils.to_categorical(valid_labels) #, num_classes=15)
print(np.shape(valid_labels))
#retraining 

# loss_fn = keras.losses.CategoricalCrossentropy()
inception_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='categorical_crossentropy', #loss_fn,
                     metrics=['accuracy', 'Precision', 'Recall'])


history = inception_model.fit_generator(train_generatorI, 
                            validation_data=(valid_images, valid_labels),
                            epochs =50)